In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import os
sys.path.insert(0,"/home/daqtest/Processor/sandpro")
import sandpro
import configparser
import json
import scipy.stats
from scipy.optimize import curve_fit
from matplotlib.colors import LogNorm
import datetime
import pandas as pd
# %run ../WaveformProcessor.py
# %run ../FastProcessing.py
# %run ../FitSPE.py

sys.path.insert(0,"../src/")
import common.d2d as d2d
import common.utils as util
import common.config_reader as common_config_reader
import data_processing.waveform_processor as waveform_processor
import data_processing.fast_processor as fast_processor
import common.metadata_handler as metadata_handler
import common.metadata_handler as metadata_handler


### Check which datasets are available

In [ ]:
# data_folder = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/test_20240926_5_T102_49V_10.0sig/"
# set_name = "config_all_20240926_183155"
# processing_mode = "all_channels" # from argument or metadata # NEW

data_folder = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241018_all_3_T98_all_voltages_3.0sig/"
set_name = "config_all_20241018_135918"
processing_mode = "all_channels" # from argument or metadata # NEW

# data_folder = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/test_20240924_T102_49V_4.0sig/"
# set_name = "config_all_20240924_193041"
# processing_mode = "all_channels" # from argument or metadata # NEW


# data_folder = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20240924_T102_48V_5.5sig/"
# set_name = "config_0_2339_20240924_122936"
# processing_mode = "single_channel" # from argument or metadata # NEW


meta_data_file = glob.glob(f"{data_folder}/meta_{set_name}.json")
bin_data_file = glob.glob(f"{data_folder}/{set_name}*.bin")

print(f"Found {len(bin_data_file)} binary data file: \n{bin_data_file}")
print(f"Found {len(meta_data_file)} meta data file: \n{meta_data_file}")

with open(meta_data_file[0], "r") as file:
    meta_data = json.load(file)


In [ ]:
common_cfg_reader = common_config_reader.ConfigurationReader()
config = common_cfg_reader.get_data_processing_config()

In [ ]:

board_number = 0 # from meta data or file name # NEW

number_of_events = 10000 # from argument or metadata # NEW
record_length_sample = int(meta_data["record_length"]) # from argument or metadata # NEW
truncate_event_front = int(config.get("EVENT_PROCESSOR_SETTINGS", "truncate_event_front")) #index to truncate the events before
truncate_event_back = int(config.get("EVENT_PROCESSOR_SETTINGS", "truncate_event_back")) #index to truncate the events before
start_index, end_index = truncate_event_front, number_of_events - truncate_event_back -1 #first 1000 events are noisy # the last 500 events might be empty



baseline_n_samples = int(config.get("EVENT_PROCESSOR_SETTINGS", "n_baseline_samples")) #index to truncate the events before
baseline_n_samples_avg = int(config.get("EVENT_PROCESSOR_SETTINGS", "n_baseline_samples_avg")) #index to truncate the events before

# get integral window to compute area
tmp = config.get("EVENT_PROCESSOR_SETTINGS", "integral_window_for_area")
tmp = tmp.split(' ')
# integral_window = (0.0, 0.6)
integral_window = (tmp[0], tmp[1])
        
# get number of channels per events
if processing_mode == "single_channel":
    n_channels = 1
elif processing_mode == "all_channels":
    # NEW
    if board_number == 0: # NEW
        n_channels = 12 # NEW
    else:  # NEW
        n_channels = 12 # NEW
        # NEW: removed processing mode as integer (between 0 and 24)
        

data_taking_config = common_cfg_reader.get_data_taking_config()

DAQ = data_taking_config.get("DAQ_MODULE", "DAQ") # which DAQ board

n_bits = int(data_taking_config.get(DAQ, "n_bits"))
range = float(data_taking_config.get(DAQ, "range"))

volt_per_adc = range/float(2**n_bits)
            
waveform = None

process_config = {"nchs": int(n_channels),
                "nsamps": int(record_length_sample),
                "sample_selection": int(baseline_n_samples), 
                "samples_to_average": int(baseline_n_samples_avg)}

# dump the config to a json file
sandpro_process_config_fname = common_cfg_reader.get_sandpro_process_config_path()
with open(sandpro_process_config_fname, "w") as f:
    json.dump(process_config, f, indent=4)
    

processor= sandpro.processing.rawdata.RawData(config_file = sandpro_process_config_fname,
                                            perchannel=False) # what does this perchannel mean?


In [ ]:
file_dir = os.path.dirname(bin_data_file[0])

baseline_std_V_list = []
baseline_mean_V_list = []
areas_Vns_list = []
heights_V_list = []
randomly_selected_raw_WF_V_list = []
randomly_selected_filtered_WF_list = []

n_selection = 100

for bin_full_path in bin_data_file:
    
    waveform = processor.get_rawdata_numpy(n_evts=number_of_events-1,
                            file=bin_full_path, # specific .bin file
                            bit_of_daq=n_bits,
                            headersize=4,inversion=False)
    
    
    for channel in np.arange(n_channels,dtype=int):
        # print(waveform["data_per_channel"][start_index:end_index,i,:].shape)
        wfp = waveform_processor.WFProcessor(file_dir, 
                                                length_per_event = record_length_sample,
                                                volt_per_adc=volt_per_adc)
        
        data_processed = waveform["data_per_channel"][start_index:end_index,channel,:]
        
        wfp.set_data(data_processed, in_adc = False)
        wfp.process_wfs()
        
        baseline_std_V = np.mean(wfp.baseline_std_V)
        baseline_mean_V = np.mean(wfp.baseline_mean_V)
        
        n_processed_events = int(len(wfp.baseline_std_V))
        start_index = int(start_index)
        
        areas_Vns = wfp.get_area(sum_window=integral_window)
        heights_V = wfp.get_height(search_window=integral_window)
        
        baseline_std_V_list.append(baseline_std_V)
        baseline_mean_V_list.append(baseline_mean_V)
        areas_Vns_list.append(areas_Vns)
        heights_V_list.append(heights_V)
        


        all_rows_id = np.arange(n_processed_events)
        selected_rows_id = np.random.choice(all_rows_id, size=n_selection, replace=False)
        # is because np.random.randint do not have replace option

        randomly_selected_raw_WF_V_list.append(data_processed[selected_rows_id,:])
        randomly_selected_filtered_WF_list.append(wfp.filtered_wfs[selected_rows_id,:])

        result_dict = {"baseline_std_V": baseline_std_V_list,
                        "baseline_mean_V": baseline_mean_V_list,
                        "areas_Vns": areas_Vns_list,
                        "heights_V": heights_V_list,
                        "randomly_selected_raw_WF_V": randomly_selected_raw_WF_V_list,
                        "randomly_selected_filtered_WF": randomly_selected_filtered_WF_list}
        


In [ ]:
board_1_time = waveform['microseconds']/1e6
board_1_time

In [ ]:
board_0_time = waveform['microseconds']/1e6

In [ ]:
for time_0, time1 in zip(board_0_time,board_1_time):
    print(time_0, time1)

In [ ]:
# threshold_mv = util.adc_to_mv(int(threshold_adc))
# # set the threshold to be 2 * Vpp
# optimal_threshold_mv =  1000 * (baseline_mean_V + 2 * (2 * np.sqrt(2) * baseline_std_V))
# optimal_threshold_3sig_mv =   1000 * (baseline_mean_V + 3 * baseline_std_V)
# optimal_threshold_adc = util.mv_to_adc(optimal_threshold_mv)

plt.close("all")

for channel in np.arange(24,dtype=int):

    figure, axes = plt.subplots(3,2,figsize=(15,15))
    # figure.subplots_adjust(wspace=0.5)
    figure.subplots_adjust(hspace=0.3)
    time = np.arange(0, record_length_sample, 1) * 4

    randomly_selected_raw_WF_V = np.transpose(randomly_selected_raw_WF_V_list[channel])
    randomly_selected_filtered_WF_V = np.transpose(randomly_selected_filtered_WF_list[channel])

    axes[0,0].set_title(f"Raw WFs channel {channel}")
    axes[0,0].plot(time, 1000 * randomly_selected_raw_WF_V,color="red",alpha=0.2)

    _ymax = 1800
    axes[0,0].set_xlim(0,time[-1]-100)
    axes[0,0].set_ylim(200,_ymax)
    axes[0,0].set_ylabel("Voltage [mV]")
    axes[0,0].set_xlabel("Time [ns]")


    # axes[0,0].axhline(optimal_threshold_mv, linestyle = '--', color='g',label=f"Optimal threshold: \n{int(optimal_threshold_adc)}[ADC] \n{int(optimal_threshold_mv)}[mV]")
    # axes[0,0].axhline(optimal_threshold_3sig_mv, linestyle = '-.', color='g',label=f"3 sig threshold: \n{int(optimal_threshold_3sig_mv)}[mV]")
    # axes[0,0].axhline(threshold_mv, color='b',label=f"Test threshold: \n{int(threshold_adc)}[ADC] \n{int(threshold_mv)}[mV]", zorder=10)

    # plot intergral window
    axes[0,0].fill_betweenx([200,_ymax], integral_window[0]*np.max(time), integral_window[1]*np.max(time), color='gray', alpha=0.5)

    # labels
    # axes[0,0].text(100,optimal_threshold_mv + _ymax/5,f"baseline mean: {1000 * baseline_mean_V:.1f} mV")
    # axes[0,0].text(100,optimal_threshold_mv + _ymax/10,f"baseline std: {1000 * baseline_std_V:.2f} mV")



    axes[0,1].set_title(f"Filtered WFs channel {channel}")
    axes[0,1].plot(time, 1000 * randomly_selected_filtered_WF_V,color="red",alpha=0.2)

    axes[0,1].set_xlim(0,time[-1]-100)
    axes[0,1].set_ylim(-5,1600)
    axes[0,1].set_ylabel("Voltage [mV]")
    axes[0,1].set_xlabel("Time [ns]")



    axes[1,0].set_title(f"Zoomed raw WFs channel {channel}")
    axes[1,0].plot(time, 1000 * randomly_selected_raw_WF_V,color="red",alpha=0.2)

    _ymax = 350
    axes[1,0].set_xlim(0,time[-1]-100)
    axes[1,0].set_ylim(200,_ymax)
    axes[1,0].set_ylabel("Voltage [mV]")
    axes[1,0].set_xlabel("Time [ns]")

    point_in_middle = time[int(len(time)/2)]
    # axes[1,0].axhline(optimal_threshold_mv, linestyle = '--', color='g')
    # axes[1,0].text(point_in_middle, optimal_threshold_mv, '~5 sigma', ha ='center', va ='center') 
    # axes[1,0].axhline(optimal_threshold_3sig_mv, linestyle = '-.', color='g')
    # axes[1,0].text(point_in_middle, optimal_threshold_3sig_mv, '3 sigma', ha ='center', va ='center') 
    # axes[1,0].axhline(threshold_mv, color='b',label=f"Test threshold: \n{int(threshold_adc)}[ADC] \n{int(threshold_mv)}[mV]", zorder=10)

    # plot intergral window
    axes[1,0].fill_betweenx([200,_ymax], integral_window[0]*np.max(time), integral_window[1]*np.max(time), color='gray', alpha=0.5)
    axes[1,0].axhline(1000 * baseline_mean_V, color="gray",linestyle="dashed",label=f"Baseline mean", zorder=10)

    # labels
    # axes[1,0].text(100,optimal_threshold_mv + _ymax/20,f"baseline mean: {1000 * baseline_mean_V:.1f} mV")
    # axes[1,0].text(100,optimal_threshold_mv + _ymax/40,f"baseline std: {1000 * baseline_std_V:.2f} mV")



    axes[1,1].set_title(f"Zoomed filtered WFs channel {channel}")
    axes[1,1].plot(time, 1000 * randomly_selected_filtered_WF_V,color="red",alpha=0.2)

    _ymax = 40
    axes[1,1].set_xlim(0,time[-1]-100)
    axes[1,1].set_ylim(-5,_ymax)
    axes[1,1].set_ylabel("Voltage [mV]")
    axes[1,1].set_xlabel("Time [ns]")


    axes[2,0].set_title(f"Integrated area distribution channel {channel}")
    axes[2,0].hist(areas_Vns_list[channel],
                    bins=200,
                    range=(-0.1, 10),
                    histtype='step',
                    color='red', 
                    label="Integrated area")
    axes[2,0].set_yscale("log")
    axes[2,0].set_ylabel("Count")
    axes[2,0].set_xlabel("Integrated Area [$V\cdot ns$]")
    axes[2,0].set_ylim(1e-1,None)

In [ ]:
# 
plt.close()

fig, axes = plt.subplots(3,8,figsize=(35,15))

for channel in np.arange(24):
    height_V = heights_V_list[channel]
    area_Vns = areas_Vns_list[channel]
    

    plot_row = channel % 3
    plot_col = channel // 3

    # change the rows so that it match with the physical layout of the channels
    if plot_row == 0:
        plot_row = 1
    elif plot_row == 1:
        plot_row = 0

    axes[plot_row,plot_col].hist2d(area_Vns,height_V,
                            bins=[100,100],
                            range=[[-0.1,800],[0,2]],
                            cmap='viridis',
                            norm=LogNorm())
    
    axes[plot_row,plot_col].set_title(f"Channel {channel}")

# set the labels
for ax in axes.flat:
    ax.set(xlabel='Area [V*ns]', ylabel='Height [V]')

# Set plot title
if data_folder[-1] == "/":
    data_folder = data_folder[:-1]
plt.suptitle(f"Dataset: {data_folder.split('/')[-1]}")

# Move title upwards
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()